In [1]:
import os, sys, cv2, pydicom
import numpy as np

In [2]:
DPATH = '/home/ubuntu/datasets/chest/CBIS-DDSM'
cases = os.listdir(DPATH)

In [3]:
import json

# with open('.cbis_cache.json', 'w') as fl:
#     json.dump(tosave, fl, indent=4)
    
with open('.cbis_cache.json') as fl:
    tosave = json.load(fl)

print(len(tosave))

3031


In [4]:
import matplotlib.pyplot as plt

In [8]:
from random import randint

def draw_sample(rawimg, rawmask, yy, xx, size=256, pad=64, case=None):
    size += pad
    half = size//2
    x0, y0 = max(0, xx-half), max(0, yy-half)
    xf, yf = min(rawimg.shape[1]-1, x0+size), min(rawimg.shape[0]-1, y0+size)
    x0, y0 = xf - size, yf - size
    
    img = rawimg[y0:yf, x0:xf]
    mask = rawmask[y0:yf, x0:xf]
    try:
        assert img.shape == (size, size)
        assert mask.shape == (size, size)
    except:
        raise Exception('Sample mismatch: %s %s' %(img.shape, mask.shape))
    return img, mask

SAVEPATH = '/home/ubuntu/datasets/tissue/cancers'
import h5py
__mismatch = 0
__toobig = []
def sample_boxes(case, ratio=2000, maxout=50):
    global __mismatch
    samples = []
    sid = case['sid']
    with pydicom.dcmread(case['image']) as db:
        raw = db.pixel_array[:]
        
    for mii, mpath in enumerate(case['masks']):
        with pydicom.dcmread(mpath) as db:
            rawmask = db.pixel_array[:]
        try:
            assert raw.shape == rawmask.shape
        except:
            # some masks have been scaled down ~10% for some reason
            scale = raw.shape[0] / rawmask.shape[0] # match Y
            rawmask = cv2.resize(rawmask, (0,0), fx=scale, fy=scale)
            scale = raw.shape[1] / rawmask.shape[1] # match X
            rawmask = cv2.resize(rawmask, (0,0), fx=scale, fy=1)
            assert raw.shape == rawmask.shape
            __mismatch += 1
            
        roiY, roiX = np.nonzero(rawmask)
        
        if len(roiX) > 320**2: 
            __toobig.append([case, mii])
            continue
#         print(len(roiX))
        samps = max(1, min(len(roiY) // ratio, maxout))
        assert samps > 0
            
        ym, xm = int(np.mean(roiY)), int(np.mean(roiX))
        img, mask = draw_sample(raw, rawmask, ym, xm, case=case)
#         if img is None: continue # some error sampling
            
        ref = '%s' % (sid)
        mask_path = '%s/%s.jpg'% (SAVEPATH, ref)
        cv2.imwrite(mask_path, mask)
        img_path = '%s/%s.npy' % (SAVEPATH, ref)
        np.save(img_path, img)
#         samples.append((img, mask))

skip = 0
selected = tosave[skip:]
# selected = [one for one in tosave if one['sid'] == '02079_RIGHT_MLO']
# assert len(selected) == 1
for ii, case in enumerate(selected):
    sample_boxes(case)
    
    sys.stdout.write('%d/%d  Miss: %d  Toobig: %d \r' % (
        ii, len(tosave)-skip, __mismatch, len(__toobig)))
    sys.stdout.flush()

In [ ]:
with open('.toobig.json', 'w') as fl:
    json.dump(__toobig, fl, indent=4)
